In [1]:
# Import classic libraries (Matplotlib and PyLab)
%matplotlib inline
%pylab inline

# Import PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Parameters (figure size and random seed)
pylab.rcParams['figure.figsize'] = (15,15)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Declare a class for MLP (multilayer perceptron)
class MLP_nn(nn.Module):
    
    # class initialization
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(MLP_nn, self).__init__()
        # fully connected layer with linear activation
        self.fc0 = nn.Linear(input_size, hidden1_size)
        # fully connected layer with linear activation
        self.fc1 = nn.Linear(hidden1_size, hidden2_size)
        # fully connected layer with linear activation
        self.fc2 = nn.Linear(hidden2_size, output_size)
        # ReLu activation
        self.relu = nn.ReLU()
        # sigmoid activation
        self.sigmoid = nn.Sigmoid()
        
    # function to apply the neural network
    def forward(self, x):
        out = self.fc0(x)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        y_pred = self.sigmoid(out)
        return y_pred
    
# Create the neural network (2 input size for x1 and x2, 4 neurons in the first hidden layer, 4 neurons in the second hidden layer, and 1 output size for y)
MLP = MLP_nn(2, 4, 4, 1)

In [ ]:
# Loss function: BCE = sum [-y log(y_pred) - (1 - y) log(1 - y_pred)]
criterion = nn.BCELoss()

# Optimizer: new_parameters = old_parameters - lr*gradient, with lr the learning rate
optimizer = torch.optim.SGD(MLP.parameters(), lr = 0.01)

In [ ]:
epochs = 50000 # number of epochs
losses = [] # list to stock the loss at each iteration

# Loop on epochs
for i in range(epochs):
    
    # compute the prediction using the previous parameters of the neural network
    y_pred = MLP.forward(torch.cat((x1, x2), dim=1))
    
    # compute and stock the loss
    loss = criterion(y_pred, y)
    losses.append(loss)
    
    # initialize the gradient to zero
    optimizer.zero_grad()
    
    # compute the gradient by back propagation
    loss.backward()
    
    # update the parameter values using the gradient
    optimizer.step()

In [ ]:
# Print the loss function
plot(range(epochs), losses)
title('Loss function', size=20)
xlabel('Epoch', size=20)
ylabel('Loss value', size=20)

In [ ]:
# Plot the results
Y_hat = MLP.forward(torch.cat((X1.reshape(10000, 1), X2.reshape(10000, 1)), dim=1))
Y_hat = Y_hat.reshape(X1.shape)
subplot(2, 2, 1)
scatter(x1, x2, c=y, cmap='bwr')
contourf(X1, X2, Y, cmap='bwr', alpha=0.5)
title('True model', size=20)
subplot(2, 2, 2)
scatter(x1, x2, c=y, cmap='bwr')
contourf(X1, X2, Y_hat.detach().numpy(), cmap='bwr', alpha=0.5)
title('Estimated model', size=20)
